In [596]:
import random
import re
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import losses
from keras import optimizers
from yaml import safe_load
from typing import List

matplotlib.use('TkAgg')

In [803]:
data = pd.read_csv("../data/cation/DataBase_RF_bi_new_2.csv")
data_mono = pd.read_csv("../data/cation/DataBase_RF_mono_new.csv")
data_admixture = pd.read_csv("../data/cation/DataBase_RF_bi_new_3.csv")

In [804]:
data_mono['name'] = data_mono['name'].str.replace(r'_[^_]*$', '', regex=True)
data_mono['name'] = data_mono['name'].str.replace(r'_[^_]*$', '', regex=True)

In [805]:
data_mono

,name,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2,S1 #1,S1' #1,S2 #1,S3 #1,S1 #2,S1' #2,S2 #2,S3 #2
0,Structure: Al_15_Si_177_Boulfelfel_Na_15,15,177,15,0,1.02,0,0.181,0,1,0,0,2,11,2,0,0,0,0
1,Structure: Al_15_Si_177_Boulfelfel_Na_15,15,177,15,0,1.02,0,0.181,0,1,0,0,2,11,2,0,0,0,0
2,Structure: Al_16_Si_176_Boulfelfel_Ca_8,16,176,8,0,1.00,0,0.472,0,2,0,0,3,2,3,0,0,0,0
3,Structure: Al_16_Si_176_Boulfelfel_Cs_16,16,176,16,0,1.67,0,2.420,0,1,0,0,3,10,3,0,0,0,0
4,Structure: Al_17_Si_175_Boulfelfel_Cs_17,17,175,17,0,1.67,0,2.420,0,1,0,0,4,10,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Structure: Al_91_Si_101_Boulfelfel_K_91,91,101,91,0,1.38,0,0.841,0,1,0,16,8,37,30,0,0,0,0
94,Structure: Al_92_Si_100_Boulfelfel_K_92,92,100,92,0,1.38,0,0.841,0,1,0,16,8,35,33,0,0,0,0
95,Structure: Al_93_Si_99_Boulfelfel_Rb_93,93,99,93,0,1.52,0,1.417,0,1,0,16,6,41,30,0,0,0,0
96,Structure: Al_94_Si_98_Boulfelfel_Ca_47,94,98,47,0,1.00,0,0.472,0,2,0,15,0,32,0,0,0,0,0


In [806]:
data = pd.concat([data, data_admixture, data_mono])

In [807]:
data = data.loc[(data["Cation #1"] == (data["S1 #1"] + data["S1\' #1"] + data["S2 #1"] + data["S3 #1"]))
                &
                (data["Cation #2"] == (data["S1 #2"] + data["S1\' #2"] + data["S2 #2"] + data[
                    "S3 #2"]))]  #["Cation #1", "S1 #1", "S1\' #1", "S2 #1", "S3 #1"]]

In [808]:
features = data.drop(columns=["S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"],
                     axis=1)
labels = data[["S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"]]

In [809]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.15,
                                                                            random_state=0)

In [810]:
from sklearn.preprocessing import MinMaxScaler
# 
# scale_features_mm = MinMaxScaler()
# features_train = scale_features_mm.fit_transform(features_train)
# features_test = scale_features_mm.transform(features_test)
# 
# scale_labels_mm = MinMaxScaler()
# labels_train = scale_labels_mm.fit_transform(labels_train)
# labels_test = scale_labels_mm.transform(labels_test)

In [811]:
def get_cations(name) -> List[str]:
    cation_list = ["K", "Na", "Cs", "Rb", "Li", "Ca"]
    res = []
    for cation in cation_list:
        if cation in name.values[0]:
            res.append(cation)
    return res


def generate_data_for_set():
    def generate(features):
        num = [[], []]
        x = [[], []]
        cation_type = []
        for i in range(len(features)):
            x[0].append(features[["Cation #1", "Radius #1", "Polarizability #1"]].iloc[i])
            x[1].append(features[["Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
            num[0].append(features[["Cation #1"]].iloc[i])
            num[1].append(features[["Cation #2"]].iloc[i])
            cation_type.append(get_cations(features[["name"]].iloc[i]))
        x[0] = np.array(x[0])
        x[1] = np.array(x[1])
        num[0] = np.array(num[0])
        num[1] = np.array(num[1])
        return x, num, cation_type

    f_train, num_train, cation_type_train = generate(features_train)
    f_test, num_test, cation_type_test = generate(features_test)

    return f_train, f_test, num_train, num_test, cation_type_train, cation_type_test


f_train, f_test, num_train, num_test, cation_type_train, cation_type_test = generate_data_for_set()

# f_train = []
# f_test = []
# num_train = []
# num_test = []
# for i in range(len(features_train)):
#     #labels_train.iloc[i] /= np.sum(labels_train.iloc[i])
#     f_train.append(features_train[["Cation #1", "Radius #1", "Polarizability #1", "Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
#     num_train.append(features_train.iloc[i]["Al"])
# for i in range(len(features_test)):
#     #labels_test.iloc[i] /= np.sum(labels_test.iloc[i])
#     f_test.append(features_test[["Cation #1", "Radius #1", "Polarizability #1", "Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
#     num_test.append(features_test.iloc[i]["Al"])

In [954]:
def mps_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    sum1 = tf.reduce_sum(y_true[:4]) + 1
    sum2 = tf.reduce_sum(y_true[4:]) + 1
    delta = tf.abs(y_pred - y_true)
    delta = tf.concat([delta[:4] / sum1, delta[4:] / sum2], axis=0)
    mean = tf.reduce_mean(delta)
    sigma = tf.math.reduce_std(delta)
    loss = mean * (1 + sigma)
    return loss






def set_model(adapt_data):
    #shared
    inputs = Input(shape=(3,), name="struct_input")
    x = Dense(3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(3, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(3, activation='relu')(x)
    feature_ext_model = Model(inputs, x)
    #
    struct_res = Input(shape=(6,))
    x = Dense(9, activation='relu')(struct_res)
    #x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(4, activation='relu')(x)
    struct_res_model = Model(struct_res, x)
    #
    
    input1, input2 = Input(shape=(3,)), Input(shape=(3,))
    normalization_layer = Normalization()
    normalization_layer.adapt(adapt_data)
    input1 = normalization_layer(input1)
    input2 = normalization_layer(input2)
    num_elements1 = Input(shape=(1,))
    num_elements2 = Input(shape=(1,))
    out1, out2 = feature_ext_model(input1), feature_ext_model(input2)
    y = Average()([out1, out2])
    struct1_state = Concatenate()([y, input1])
    struct2_state = Concatenate()([y, input2])
    struct1_state = struct_res_model(struct1_state)
    struct2_state = struct_res_model(struct2_state)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct1_state)
    output1 = Multiply()([l1, num_elements1])
    l2 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct2_state)
    output2 = Multiply()([l2, num_elements2])
    out = Concatenate()([output1, output2])
    model = Model([input1, input2, num_elements1, num_elements2], out)
    model.compile(loss='mae', optimizer=optimizers.Adam(clipnorm=0.1))
    return model


import tensorflow as tf

def set_model2(adapt_data):
    #
    struct1, struct2 = Input(shape=3), Input(shape=3)
    normalization_layer = Normalization()
    normalization_layer.adapt(adapt_data)
    struct1 = normalization_layer(struct1)
    struct2 = normalization_layer(struct2)
    inputs = Concatenate()([struct1, struct2])
    x = Dense(6, activation='relu', name="struct_d1")(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation='relu')(x)
    #x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation='relu')(x)
    struct_model = Model([struct1, struct2], x)
    #
    #
    struct_res = Input(shape=(9,))
    x = Dense(9, activation='relu', name="struct_d2")(struct_res)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(9, activation='relu')(x)
    #x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(4, activation='relu')(x)
    struct_res_model = Model(struct_res, x)
    #
    input1, input2 = Input(shape=(3,)), Input(shape=(3,))
    num_elements1 = Input(shape=(1,))
    num_elements2 = Input(shape=(1,))
    out1, out2 = struct_model([input1, input2]), struct_model([input2, input1])
    y = Average()([out1, out2])
    struct1_state = Concatenate()([y, input1])
    struct2_state = Concatenate()([y, input2])
    struct1_state = struct_res_model(struct1_state)
    struct2_state = struct_res_model(struct2_state)
    
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct1_state)
    output1 = Multiply()([l1, num_elements1])
    l2 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct2_state)
    output2 = Multiply()([l2, num_elements2])
    out = Concatenate()([output1, output2])
    
    model = Model([input1, input2, num_elements1, num_elements2], out)
    model.compile(loss='mse', optimizer=optimizers.Adam()) # losses.Huber(delta=5.0)
    return model



def create_model(x):
    inputs = Input(shape=6)
    num_elements = Input(shape=1)
    x = Dense(10, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    d = Dense(8, activation='relu')(x)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(d)
    outputs = Multiply()([l1, num_elements])
    model = tf.keras.Model(inputs=[inputs, num_elements], outputs=outputs)
    model.compile(loss="mse", optimizer=optimizers.Adam())
    return model


#model = set_model(np.concatenate((f_train[0], f_train[1])))
model = set_model2(np.concatenate((f_train[0], f_train[1])))

In [981]:
# model.compile(optimizer='adam', loss=mps_loss)
# from keras import backend as K
#K.set_value(model.optimizer.learning_rate, 0.01)
model.compile(loss=mps_loss, optimizer=optimizers.Adam())

In [982]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()


mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='../data/models/mps_old_tuned.keras', save_best_only=True,
                                              monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                              save_freq='epoch')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                      patience=1000, verbose=1, mode='auto')

# history = model.fit([np.array(f_train), np.array(num_train)], labels_train,
#                     epochs=100, batch_size=16, shuffle=True,
#                     validation_data=([np.array(f_test), np.array(num_test)], labels_test), callbacks=[mcp_save])

history = model.fit((f_train[0], f_train[1], num_train[0], num_train[1]), labels_train,
                    epochs=500, batch_size=64, shuffle=True,
                    validation_data=((f_test[0], f_test[1], num_test[0], num_test[1]), labels_test), callbacks=[mcp_save, reduce_lr_loss])

# history = model.fit((f_train[0], f_train[1], num_train[0], num_train[1]), labels_train,
#                     epochs=500, batch_size=64, shuffle=True,
#                     validation_data=((f_ref[0], f_ref[1], num_ref[0], num_ref[1]), labels_ref), callbacks=[mcp_save, reduce_lr_loss])

plot_loss(history)

Epoch 1/500
34/34 [==============================] - ETA: 0s - loss: 0.0011
Epoch 1: val_loss improved from inf to 0.00112, saving model to ../data/models\mps_old_tuned.keras
34/34 [==============================] - 3s 26ms/step - loss: 0.0011 - val_loss: 0.0011 - lr: 0.0010
Epoch 2/500
33/34 [============================>.] - ETA: 0s - loss: 0.0011
Epoch 2: val_loss improved from 0.00112 to 0.00110, saving model to ../data/models\mps_old_tuned.keras
34/34 [==============================] - 1s 17ms/step - loss: 0.0012 - val_loss: 0.0011 - lr: 0.0010
Epoch 3/500
33/34 [============================>.] - ETA: 0s - loss: 0.0011
Epoch 3: val_loss did not improve from 0.00110
34/34 [==============================] - 1s 15ms/step - loss: 0.0011 - val_loss: 0.0011 - lr: 0.0010
Epoch 4/500
33/34 [============================>.] - ETA: 0s - loss: 0.0011
Epoch 4: val_loss did not improve from 0.00110
34/34 [==============================] - 1s 15ms/step - loss: 0.0011 - val_loss: 0.0011 - lr: 0.0

In [983]:
model = tf.keras.models.load_model('../data/models/mps_old_tuned.keras', custom_objects={'mps_loss': mps_loss})

In [984]:
def process_prediction(prediction, num1, num2):
    return np.concatenate((prediction[:4] * num1, prediction[4:] * num2))


#predictions = model.predict([np.array(f_test), np.array(num_test)])
predictions = model.predict((f_test[0], f_test[1], num_test[0], num_test[1]))
predictions_2 = model.predict((f_test[1], f_test[0], num_test[1], num_test[0]))

#predictions = [process_prediction(predictions[i], num_test[0][i], num_test[1][i]) for i in range(len(predictions))]
#labels_test = [process_prediction(labels_test[i], num_test[0][i], num_test[1][i]) for i in range(len(labels_test))]

12/12 [==============================] - 0s 4ms/step


In [985]:
# print(features_test.iloc[i])
# print(labels_test.iloc[i])

In [986]:
def mps(prediction:np.array, reference:np.array, is_mono=False):
    is_mono = (np.sum(reference[4:]) == 0)
    if not is_mono:
        sum1 = np.sum(reference[:4])
        sum2 = np.sum(reference[4:])
        delta = np.abs(prediction - reference)
        delta = np.concatenate((delta[:4]/sum1, delta[4:]/sum2))
        mean = np.mean(delta)
        sigma = np.std(delta)
        return mean * (1 + sigma)
    else:
        prediction = prediction[:4]
        reference = reference[:4]
        sum = np.sum(reference)
        delta = np.abs(prediction - reference)/sum
        mean = np.mean(delta)
        sigma = np.std(delta)
        return mean * (1 + sigma)


i = random.randint(0, len(predictions) - 1)
print("№", i)
rounded_prediction = [round(p, 1) for p in predictions[i]]
print(rounded_prediction)
rounded_prediction2 = [round(p, 1) for p in predictions_2[i]]
#print(rounded_prediction2)

rounded_real = [round(float(p), 1) for p in np.array(labels_test.iloc[i])]
print(rounded_real)
print("mps:", mps(prediction=predictions[i], reference=labels_test.iloc[i]))
print("R2:", metrics.r2_score(y_true=labels_test.iloc[i], y_pred=predictions[i]))
print(sum(predictions[i]), sum(labels_test.iloc[i]))

№ 281
[1.2, 2.8, 12.5, 2.4, 0.1, 1.3, 6.0, 1.6]
[3.0, 0.0, 15.0, 1.0, 0.0, 1.0, 6.0, 2.0]
mps: 0.0695409641415634
R2: 0.8911378244928007
27.999999679625034 28


In [987]:
# ref1 = [0.0, 5.5, 25.1, 21.5, 9.8, 9.282, 10.283, 9.555]
# s1 = [6.28166667,  6.38166667, 12.23666667, 25.4275, 1.01583333, 12.13, 20.70583333,  4.2125]
# metrics.r2_score(y_true=ref1, y_pred=s1)

In [988]:
mps_list_bi = []
mps_mono = []
mps_list = []
for i in range(len(predictions)):
    p = predictions[i]
    real = np.array(labels_test.iloc[i])
    if num_test[1][i] == 0:
        mps_value = mps(prediction=p, reference=real, is_mono=True)
        mps_mono.append(mps_value)
    else:
        mps_value = mps(prediction=p, reference=real, is_mono=False)
        mps_list_bi.append(mps_value)
    mps_list.append(mps_value)
print(sum(mps_list_bi)/len(mps_list_bi), sum(mps_mono)/len(mps_mono))

0.13728131815671352 0.05996253995703056


In [989]:
metrics.r2_score(y_true=labels_test, y_pred=predictions)

0.8366428393315788

In [925]:
plt.hist(mps_list_bi)
plt.show()

In [926]:
plt.scatter([str(i) for  i in cation_type_test], mps_list)
plt.show()

In [757]:
reference_data = pd.read_csv('../data/cation/Reference_Data_Base_2024_fp.csv', delimiter=',', header=None,)
reference_data = reference_data.T
new_header = reference_data.iloc[0]
reference_data = reference_data[1:]
reference_data.columns = new_header
reference_data

,Cation #1,Number of Cations #1 per UC,Charge of Cation #1,Cation #2,Number of Cations #2 per UC,Charge of Cation #2,Cation #3,Number of cations #3 per UC,Charge of Cation #3,Number of Al atoms per UC,...,Charge balance (True/False),Data source (NMR/XRD),Dehydrated (yes/no),Dehydration temperature (in K),Evacuated (yes/no),Evacuation pressure (in Torr),Crystallography structure (yes/no),Average UC length (in Angstroms),Doi:,Comments
1,K,48.2,1,-,-,-,-,-,-,48.2,...,True,XRD,yes,573,yes,1.00E-5,yes,24.776,https://doi.org/10.1021/j100649a008,NaN
2,K,54.7,1,-,-,-,-,-,-,54.7,...,True,XRD,yes,573,yes,1.00E-5,yes,24.896,https://doi.org/10.1021/j100649a008,NaN
3,K,69.8,1,-,-,-,-,-,-,69.8,...,True,XRD,yes,573,yes,1.00E-5,yes,24.973,https://doi.org/10.1021/j100649a008,NaN
4,K,86.5,1,-,-,-,-,-,-,86.5,...,True,XRD,yes,573,yes,1.00E-5,yes,25.076,https://doi.org/10.1021/j100649a008,NaN
5,Na,93,1,-,-,-,-,-,-,93,...,True,NMR,yes,673,yes,8.00E-5,no,-,https://doi.org/10.1021/ja0102538,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Ca,28,2,NaN,NaN,NaN,NaN,NaN,NaN,56.0,...,True,XRD,yes,748,no,NaN,yes,24.71,https://doi.org/10.1016/0025-5408(68)90112-8,NaN
107,Ca,25,2,NaN,NaN,NaN,NaN,NaN,NaN,50.0,...,True,XRD,yes,520,yes,7.50E-6,no,24.7,https://doi.org/10.1016/S0167-2991(08)64089-6,NaN
108,Sr,46,2,NaN,NaN,NaN,NaN,NaN,NaN,92.0,...,True,XRD,yes,633,yes,2.00E-6,no,25.214,https://doi.org/10.1016/S1387-1811(99)00012-8,NaN
109,Sr,32,2,Rb,16,1,Na,4,1,84.0,...,True,XRD,yes,573,no,NaN,no,25.15,https://doi.org/10.1016/S0167-2991(04)80689-X,11.2 molecules H2O per UC


In [758]:
bi_cation_data = reference_data.loc[(reference_data["Cation #1"].notnull()) & (reference_data["Cation #1"] != "-") &
                          (reference_data["Cation #2"].notnull()) & (reference_data["Cation #2"] != "-") &
                          (reference_data["Cation #3"].isnull() | (reference_data["Cation #3"] == "-"))]
bi_cation_data

,Cation #1,Number of Cations #1 per UC,Charge of Cation #1,Cation #2,Number of Cations #2 per UC,Charge of Cation #2,Cation #3,Number of cations #3 per UC,Charge of Cation #3,Number of Al atoms per UC,...,Charge balance (True/False),Data source (NMR/XRD),Dehydrated (yes/no),Dehydration temperature (in K),Evacuated (yes/no),Evacuation pressure (in Torr),Crystallography structure (yes/no),Average UC length (in Angstroms),Doi:,Comments
16,K,52,1,Na,39,1,NaN,NaN,NaN,91.0,...,True,XRD,yes,673,no,760,NaN,24.861,https://doi.org/10.1016/s0144-2449(05)80234-9,NaN
17,Cs,45,1,Na,47,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,673,yes,2.00E-6,yes,24.898,https://doi.org/10.1016/j.micromeso.2004.01.006,NaN
19,Co,38.1,2,Na,15.8,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,673,yes,1.00E-3,yes,24.597,https://doi.org/10.1016/j.micromeso.2007.12.031,NaN
20,Cs,45.9,1,Na,37.6,1,NaN,NaN,NaN,83.5,...,True,NMR,yes,673,yes,7.50E-5,no,NaN,https://doi.org/10.1021/jp001571g,NaN
21,Ca,19,2,Na,48,1,NaN,NaN,NaN,86.0,...,True,XRD,yes,773,yes,NaN,no,NaN,https://doi.org/10.1016/j.micromeso.2007.01.043,NaN
24,Co,34,2,Ca,9,2,NaN,NaN,NaN,86.0,...,True,XRD,yes,773,no,NaN,NaN,NaN,https://doi.org/10.1016/j.micromeso.2007.01.043,NaN
34,Cs,44,1,Na,47,1,NaN,NaN,NaN,91.0,...,True,XRD,yes,673,no,7.60E+2,yes,24.98,10.1016/s0144-2449(05)80234-9,NaN
36,Li,86,1,Na,6,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,623,yes,1.00E-6,NaN,NaN,https://doi.org/10.7745/KJSSF.2013.46.4.260,NaN
37,Li,87,1,Na,5,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,623,yes,1.00E-6,NaN,NaN,https://doi.org/10.7745/KJSSF.2013.46.4.261,NaN
38,Li,81,1,Na,15,1,NaN,NaN,NaN,96.0,...,True,NMR,yes,296,NaN,7.50E-5,NaN,NaN,https://doi.org/10.1016/S1387-1811(98)00211-X,NaN


In [759]:
mono_cation_data = reference_data.loc[(reference_data["Cation #1"].notnull()) & (reference_data["Cation #1"] != "-") &
                          (reference_data["Cation #2"].isnull()) | (reference_data["Cation #2"] == "-") &
                          (reference_data["Cation #3"].isnull() | (reference_data["Cation #3"] == "-"))]
mono_cation_data = mono_cation_data.fillna(0)
mono_cation_data = mono_cation_data.replace("-", 0)
mono_cation_data

C:\Users\ivano\AppData\Local\Temp\ipykernel_11484\1448643730.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mono_cation_data = mono_cation_data.replace("-", 0)


,Cation #1,Number of Cations #1 per UC,Charge of Cation #1,Cation #2,Number of Cations #2 per UC,Charge of Cation #2,Cation #3,Number of cations #3 per UC,Charge of Cation #3,Number of Al atoms per UC,...,Charge balance (True/False),Data source (NMR/XRD),Dehydrated (yes/no),Dehydration temperature (in K),Evacuated (yes/no),Evacuation pressure (in Torr),Crystallography structure (yes/no),Average UC length (in Angstroms),Doi:,Comments
1,K,48.2,1,0,0,0,0,0,0,48.2,...,True,XRD,yes,573,yes,1.00E-5,yes,24.776,https://doi.org/10.1021/j100649a008,0
2,K,54.7,1,0,0,0,0,0,0,54.7,...,True,XRD,yes,573,yes,1.00E-5,yes,24.896,https://doi.org/10.1021/j100649a008,0
3,K,69.8,1,0,0,0,0,0,0,69.8,...,True,XRD,yes,573,yes,1.00E-5,yes,24.973,https://doi.org/10.1021/j100649a008,0
4,K,86.5,1,0,0,0,0,0,0,86.5,...,True,XRD,yes,573,yes,1.00E-5,yes,25.076,https://doi.org/10.1021/j100649a008,0
5,Na,93,1,0,0,0,0,0,0,93,...,True,NMR,yes,673,yes,8.00E-5,no,0,https://doi.org/10.1021/ja0102538,0
6,Na,83,1,0,0,0,0,0,0,83,...,True,NMR,yes,673,yes,8.00E-5,no,0,https://doi.org/10.1021/ja0102538,0
7,Na,53,1,0,0,0,0,0,0,53.0,...,True,NMR,yes,673,yes,8.00E-5,no,0,https://doi.org/10.1021/ja0102538,0
8,Na,88,1,0,0,0,0,0,0,88.0,...,True,XRD,yes,673,yes,0,yes,25.09153,https://doi.org/10.1021/jp054636u,J_site - last cations
9,Na,88,1,0,0,0,0,0,0,88.0,...,True,XRD,yes,673,yes,2.00E-6,yes,25.099,https://doi.org/10.1016/0144-2449(95)00029-6,Cations separated in 3 and 1 sites
10,Na,92,1,0,0,0,0,0,0,92.0,...,True,XRD,yes,673,yes,1.00E-5,yes,0,https://doi.org/10.1021/jp991817l,0


In [760]:
cation_db = pd.read_csv('../data/cation/cations_params.csv')
cation_db

,Cation,Radius,Polarizability
0,Li,0.76,0.029
1,Na,1.02,0.181
2,K,1.38,0.841
3,Rb,1.52,1.417
4,Cs,1.67,2.420
5,Ag,1.15,1.360
6,Tl,1.50,0.940
7,Ca,1.00,0.472
8,Mg,0.72,0.094
9,Ba,1.35,1.563


In [761]:
def generate_data_for_reference(data_ref: pd.DataFrame, cation_db: pd.DataFrame, is_mono):
    num_ref = [[], []]
    f_ref = [[], []]
    labels_ref = []
    cations_type = []
    
    for index, row in data_ref.iterrows():
        cation_number1 = float(row["Number of Cations #1 per UC"])
        cation_number2 = float(row["Number of Cations #2 per UC"])
        num_ref[0].append(cation_number1)
        num_ref[1].append(cation_number2)
        radius1 = cation_db[cation_db["Cation"] == row["Cation #1"]]["Radius"].values[0]
        radius2 = 0 if is_mono else cation_db[cation_db["Cation"] == row["Cation #2"]]["Radius"].values[0]
        polarizability1 = cation_db[cation_db["Cation"] == row["Cation #1"]]["Polarizability"].values[0]
        polarizability2 = 0 if is_mono else cation_db[cation_db["Cation"] == row["Cation #2"]]["Polarizability"].values[0]
        f_ref[0].append([cation_number1, radius1, polarizability1])
        f_ref[1].append([cation_number2, radius2, polarizability2])
        
        labels_ref.append([row["SI site occupation by Cation #1"], row["SI\' site occupation by Cation #1"],
                           row["SII site occupation by Cation #1"], row["SIII site occupation by Cation #1"],
                           
                           row["SI site occupation by Cation #2"], row["SI\' site occupation by Cation #2"],
                           row["SII site occupation by Cation #2"], row["SIII site occupation by Cation #2"]])
        cations_type.append([row["Cation #1"], row["Cation #2"]])
    f_ref[0] = np.array(f_ref[0])
    f_ref[1] = np.array(f_ref[1])
    num_ref[0] = np.array(num_ref[0])
    num_ref[1] = np.array(num_ref[1])
    labels_ref = np.array(labels_ref)
    return f_ref, num_ref, labels_ref, cations_type

f_ref, num_ref, labels_ref, cations_type = generate_data_for_reference(bi_cation_data, cation_db, is_mono=False)
f_ref_mono, num_ref_mono, labels_ref_mono, cations_type_mono = generate_data_for_reference(mono_cation_data, cation_db, is_mono=True)
labels_ref = labels_ref.astype(float)
labels_ref_mono = labels_ref_mono.astype(float)

In [990]:
predictions_ref = model.predict((f_ref[0], f_ref[1], num_ref[0], num_ref[1]))
predictions_ref_mono = model.predict((f_ref_mono[0], f_ref_mono[1], num_ref_mono[0], num_ref_mono[1]))

2/2 [==============================] - 0s 6ms/step


In [976]:
# find_lst = []
# for i, c in enumerate(cations_type):
#         if c == ["Ca", "Na"]:
#             find_lst.append(i)
# find_lst

In [977]:
def analyze_random_prediction(prediction, label, cation_t):
    i = random.randint(0, len(prediction) - 1)
    print("№", i)
    rounded_prediction = [round(p, 1) for p in prediction[i]]
    print(rounded_prediction)
    rounded_real = [round(float(p), 1) for p in np.array(label[i])]
    print(rounded_real)
    print("mps: ", mps(prediction[i], label[i]))
    print(cation_t[i])
#analyze_random_prediction(predictions_ref, labels_ref, cations_type)
analyze_random_prediction(predictions_ref_mono, labels_ref_mono, cations_type_mono)

№ 31
[5.0, 6.7, 28.0, 6.4, 0.0, 0.0, 0.0, 0.0]
[16.0, 0.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mps:  0.1517118721451252
['Sr', 0]


In [991]:
mps_list = []
for i in range(len(predictions_ref)):
    p = predictions_ref[i]
    real = np.array(labels_ref[i])
    mps_list.append(mps(prediction=p, reference=real))
print(sum(mps_list))
print(sum(mps_list)/len(mps_list))
plt.hist(mps_list)
plt.show()

11.017993158982994
0.21603908154868617


In [944]:
plt.scatter([str(i) for  i in cations_type], mps_list)
plt.show()

In [992]:
mps_list = []
for i in range(len(predictions_ref_mono)):
    p = predictions_ref_mono[i]
    real = np.array(labels_ref_mono[i])
    mps_list.append(mps(prediction=p, reference=real))
print(sum(mps_list))
print(sum(mps_list)/len(mps_list))
plt.hist(mps_list)
plt.show()

7.0423248382922905
0.12354955856653141


In [946]:
plt.scatter([str(i) for  i in cations_type_mono], mps_list)
plt.show()